## Analisando e tratando os dados do arquivo Sales.Customer.csv que estão na pasta RAW
### Ao final desse notebook os dados serão carregados na pasta REFINED em formato parquet

In [1]:
# importando as bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
# criando a variavel com o caminho do arquivo que iremos carregar
path_customer = "gs://bike-factory-datalake/01.RAW/Sales.Customer.csv"

In [3]:
# carregando o arquivo CSV
customer = pd.read_csv(path_customer,sep=';')

In [4]:
# criando o DF pandas
df_customer = pd.DataFrame(customer)

In [5]:
df_customer.dtypes

CustomerID         int64
PersonID         float64
StoreID          float64
TerritoryID        int64
AccountNumber     object
rowguid           object
ModifiedDate      object
dtype: object

In [6]:
# verificando as primeiras linhas do DF
df_customer.head(100)

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
...,...,...,...,...,...,...,...
95,96,NaN,1308.0,4,AW00000096,BAC5B4DA-00B0-4BF1-816A-1E39F676BF9B,2014-09-12 11:15:07.263
96,97,NaN,1254.0,4,AW00000097,355EB5D8-6B4C-4FFF-B12B-40F89CA4E31D,2014-09-12 11:15:07.263
97,98,NaN,734.0,5,AW00000098,7D37485D-A4CF-4C08-8D3C-364DEEC7C841,2014-09-12 11:15:07.263
98,99,NaN,1310.0,5,AW00000099,8583A63E-3459-44C7-8736-A92E0A7135F2,2014-09-12 11:15:07.263


In [7]:
# Coletando informações básicas do DF
# A saída desse comando será a quantidade de linhas e colunas (linhas, colunas)
df_customer.shape

(19820, 7)

In [8]:
# verificando o index do arquivo
df_customer.index

RangeIndex(start=0, stop=19820, step=1)

In [9]:
# Verificando as colunas do DF
df_customer.columns

Index(['CustomerID', 'PersonID', 'StoreID', 'TerritoryID', 'AccountNumber',
       'rowguid', 'ModifiedDate'],
      dtype='object')

In [10]:
# Contagem de dados não nulos
df_customer.count()

CustomerID       19820
PersonID         19119
StoreID           1336
TerritoryID      19820
AccountNumber    19820
rowguid          19820
ModifiedDate     19820
dtype: int64

In [11]:
# Identificando a quantidade de dados nulos em cada coluna
df_customer.isnull().sum()

CustomerID           0
PersonID           701
StoreID          18484
TerritoryID          0
AccountNumber        0
rowguid              0
ModifiedDate         0
dtype: int64

Nesse DataFrame vemos os seguintes problemas para serem tratados:

- Coluna PersonID, conforme o modelo relacional fornecido, é uma FK na tabela Person.Person, portanto, as linhas nulas não nos interessam, pois não teremos como fazer o relacionamento, iremos dropar todas as linhas nulas e como não faremos nenhuma operação matemática com essa coluna, devemos analisar se mudamos o tipo dela para INT ou STR.
- A Coluna StoreID também foi reconhecida possui 1.336 linhas válidas e para responder as perguntas, essa coluna não tem valor, então iremos dropar.
- A Coluna ModifiedDate está com o tipo Object e deverá ser trocado para o tipo date.

In [12]:
# Limpando os dados nulos da coluna PersonID
df_customer = df_customer.dropna(subset=['PersonID'])

In [13]:
# Dropando a coluna StoreID
df_customer = df_customer.drop('StoreID',axis=1)

In [14]:
# Verificando as quantidades de linhas
df_customer.count()

CustomerID       19119
PersonID         19119
TerritoryID      19119
AccountNumber    19119
rowguid          19119
ModifiedDate     19119
dtype: int64

Por fim, já temos todas as nossas colunas tratadas, então iremos salvar o DF na pasta REFINED do nosso Lake em um arquivo parquet que é otimizado para o modelo analítico.

In [15]:
df_customer.to_parquet('gs://bike-factory-datalake/02.REFINED/sales.customer.parquet')